# Lab 3
#### 学生信息：
学号：24320172205050

姓名：博雅力

# Apriori

In [1]:
import os
from tqdm import tqdm

## 3-1 (Infaraway)

In [2]:
from Apriori1 import *

def loaddat(file_name, flag = ' '):
    dataSetDict = {}
    dataSet = []
    num_lines = sum(1 for line in open(file_name,'r'))
    
    print("reading data from", file_name)
    
    with open(file_name,'r') as f:
        for line in tqdm(f, total=num_lines):
            line = line.strip().split(flag)
            dataSet.append(line)
            dataLine = [word for word in line]
            dataSetDict[frozenset(dataLine)] = dataSetDict.get(frozenset(dataLine), 0) + 1
    
    print("done")

    return dataSetDict, dataSet, num_lines

In [3]:
def test_apriori(minSup, dataSetDict, dataSet):
    freqItems = apriori_zc(dataSet, dataSetDict, minSup)
    freqItems = sorted(freqItems.items(), key=lambda item: item[1])
    return sorted(freqItems)

In [4]:
kosarak_data_file = os.path.join('kosarak.dat')

In [5]:
dataSetDict, dataSet, num_lines = loaddat(kosarak_data_file)

  3%|▎         | 27885/990002 [00:00<00:03, 278849.60it/s]

reading data from kosarak.dat


100%|██████████| 990002/990002 [00:05<00:00, 165971.93it/s]

done


In [6]:
minSup = 5

In [7]:
freqItems_ap = test_apriori(minSup, dataSetDict, dataSet)

In [8]:
print("minSup :", minSup, "data_num :", num_lines ,"freqItems_ap:", len(freqItems_ap))

minSup : 5 data_num : 990002 freqItems_ap: 0


In [9]:
min_sup = 0.2 * num_lines
freqItems = apriori_zc(dataSet, dataSetDict, 0.1)
freqItems = sorted(freqItems.items(), key=lambda item: item[1], reverse=True)
max = 0
index = 0

for i, item in enumerate(freqItems):
    if len(item[0]) > max:
        max = len(item[0])
        index = i
        


In [10]:
print(freqItems)

[(frozenset({'1'}), 1), (frozenset({'2'}), 1), (frozenset({'3'}), 1)]


In [11]:
import os
from tqdm import tqdm

In [12]:
kosarak_data_file = os.path.join('kosarak.dat')
num_lines = sum(1 for line in open(kosarak_data_file,'r'))

Quite hard to use implementation from Infaraway, let's try some other implementations

## 3-2

In [13]:
print('reading from file: ')
loaded_list_list = []

with open(kosarak_data_file,'r') as f:
    for line in tqdm(f, total=num_lines):
        temp_list = []
#         temp_tup = (word for word in line)
        for x in line:
            if x != '\n' and x != ' ':
                temp_list.append(x)
        loaded_list_list.append(temp_list)
        
print('Done!')

  0%|          | 0/990002 [00:00<?, ?it/s]

reading from file: 


100%|██████████| 990002/990002 [00:06<00:00, 148846.14it/s]

Done!


In [14]:
from  Apriori import *

In [15]:
C1 = createC1(loaded_list_list)
C1

[frozenset({'0'}),
 frozenset({'1'}),
 frozenset({'2'}),
 frozenset({'3'}),
 frozenset({'4'}),
 frozenset({'5'}),
 frozenset({'6'}),
 frozenset({'7'}),
 frozenset({'8'}),
 frozenset({'9'})]

In [16]:
D = list(map(set,loaded_list_list))

In [17]:
L1, suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({'6'}),
 frozenset({'4'}),
 frozenset({'3'}),
 frozenset({'2'}),
 frozenset({'1'})]

In [18]:
%%time
L, suppData = apriori(loaded_list_list, minSupport=0.5)
rules = generateRules(L, suppData, minConf=0.7)

frozenset({'6'}) --> frozenset({'1'}) conf: 0.7868650084175777
frozenset({'1'}) --> frozenset({'6'}) conf: 0.7957586387427283
frozenset({'6'}) --> frozenset({'3'}) conf: 0.7490765381647209
frozenset({'3'}) --> frozenset({'6'}) conf: 0.7599111241128083
frozenset({'3'}) --> frozenset({'1'}) conf: 0.761323665642783
frozenset({'1'}) --> frozenset({'3'}) conf: 0.7589511997655878
CPU times: user 7.83 s, sys: 215 ms, total: 8.05 s
Wall time: 8.08 s


## Efficient Apriori
**Source:** [tommyod](https://github.com/tommyod/Efficient-Apriori)


    The classic apriori algorithm as described in 1994 by Agrawal et al.
    
    The Apriori algorithm works in two phases. Phase 1 iterates over the 
    transactions several times to build up itemsets of the desired support
    level. Phase 2 builds association rules of the desired confidence given the
    itemsets found in Phase 1. Both of these phases may be correctly
    implemented by exhausting the search space, i.e. generating every possible
    itemset and checking it's support. The Apriori prunes the search space
    efficiently by deciding apriori if an itemset possibly has the desired
    support, before iterating over the entire dataset and checking.
    
    Parameters
    ----------
    transactions : list of tuples, list of itemsets.TransactionWithId,
        or a callable returning a generator. Use TransactionWithId's when
        the transactions have ids which should appear in the outputs.
        The transactions may be either a list of tuples, where the tuples must
        contain hashable items. Alternatively, a callable returning a generator
        may be passed. A generator is not sufficient, since the algorithm will
        exhaust it, and it needs to iterate over it several times. Therefore,
        a callable returning a generator must be passed.
    min_support : float
        The minimum support of the rules returned. The support is frequency of
        which the items in the rule appear together in the data set.
    min_confidence : float
        The minimum confidence of the rules returned. Given a rule X -> Y, the
        confidence is the probability of Y, given X, i.e. P(Y|X) = conf(X -> Y)
    max_length : int
        The maximum length of the itemsets and the rules.
    verbosity : int
        The level of detail printing when the algorithm runs. Either 0, 1 or 2.
    output_transaction_ids : bool
        If set to true, the output contains the ids of transactions that
        contain a frequent itemset. The ids are the enumeration of the
        transactions in the sequence they appear.
    Examples
    --------
    >>> transactions = [('a', 'b', 'c'), ('a', 'b', 'd'), ('f', 'b', 'g')]
    >>> itemsets, rules = apriori(transactions, min_confidence=1)
    >>> rules
    [{a} -> {b}]

In [19]:
import typing
from efficient_apriori.itemsets import itemsets_from_transactions, ItemsetCount, generate_rules_apriori
# from efficient_apriori.rules import generate_rules_apriori

# >>> transactions = [('a', 'b', 'c'), ('a', 'b', 'd'), ('f', 'b', 'g')]
# >>> itemsets, rules = apriori(transactions, min_confidence=1)

def apriori(
    transactions: typing.Union[typing.List[tuple], typing.Callable],
    min_support: float = 0.5,
    min_confidence: float = 0.5,
    max_length: int = 8,
    verbosity: int = 0,
    output_transaction_ids: bool = False,
):

    itemsets, num_trans = itemsets_from_transactions(
        transactions,
        min_support,
        max_length,
        verbosity,
        output_transaction_ids,
    )

    if itemsets and isinstance(next(iter(itemsets[1].values())), ItemsetCount):
        itemsets_for_rules = _convert_to_counts(itemsets)
    else:
        itemsets_for_rules = itemsets

    rules = generate_rules_apriori(
        itemsets_for_rules, min_confidence, num_trans, verbosity
    )
    return itemsets, list(rules)


def _convert_to_counts(itemsets):
    itemsets_counts = {}
    for size, sets in itemsets.items():
        itemsets_counts[size] = {i: c.itemset_count for i, c in sets.items()}
    return itemsets_counts

#### Testing algorithm:

In [20]:
loaded_list_tuples = []

print('reading from file: ')

with open(kosarak_data_file,'r') as f:
    for line in tqdm(f, total=num_lines):
        temp_tup = ()
#         temp_tup = (word for word in line)
        for x in line:
            if x != '\n' and x != ' ':
                temp_tup = temp_tup + (x, )
        loaded_list_tuples.append(temp_tup)

  2%|▏         | 17018/990002 [00:00<00:12, 80974.32it/s]

reading from file: 


100%|██████████| 990002/990002 [00:12<00:00, 77369.00it/s]


Let's use the same arguments, so we can compare time

In [21]:
%%time
itemsets, rules = apriori(loaded_list_tuples, min_confidence=0.7)

CPU times: user 4.1 s, sys: 47 ms, total: 4.15 s
Wall time: 4.15 s


In [22]:
for x in  rules :
    print(x)

{3} -> {1} (conf: 0.761, supp: 0.552, lift: 1.047, conv: 1.142)
{1} -> {3} (conf: 0.759, supp: 0.552, lift: 1.047, conv: 1.140)
{6} -> {1} (conf: 0.787, supp: 0.579, lift: 1.082, conv: 1.279)
{1} -> {6} (conf: 0.796, supp: 0.579, lift: 1.082, conv: 1.295)
{6} -> {3} (conf: 0.749, supp: 0.551, lift: 1.033, conv: 1.096)
{3} -> {6} (conf: 0.760, supp: 0.551, lift: 1.033, conv: 1.101)


Efficient implementation is upto 100% faster, than a previous(simpler) implementation 

# FPtree

In [23]:
import pandas as pd
import numpy as np

In [24]:
# % conda install xlrd

In [25]:
df = pd.read_excel('Transactions.xls', dtype='int8')  # can be bool, harder to read though
df.head()

,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,beer,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   pasta              10000 non-null  int8 
 1   milk               10000 non-null  int8 
 2   water              10000 non-null  int8 
 3   biscuits           10000 non-null  int8 
 4   coffee             10000 non-null  int8 
 5   brioches           10000 non-null  int8 
 6   yoghurt            10000 non-null  int8 
 7   frozen vegetables  10000 non-null  int8 
 8   tunny              10000 non-null  int8 
 9   beer               10000 non-null  int8 
 10  tomato souce       10000 non-null  int8 
 11  coke               10000 non-null  int8 
 12  rice               10000 non-null  int8 
 13  juices             10000 non-null  int8 
 14  crackers           10000 non-null  int8 
 15  oil                10000 non-null  int8 
 16  frozen fish        10000 non-null  int8 
 17  ice cream    

In [27]:
allTransection = []
datalen = len(df)
totalCol = len(df.columns)

In [28]:
# totalCol

making basket

In [29]:
for i in range(0, datalen):
    allTransection.append([str(df.values[i, j]) for j in range(0, totalCol)])

In [30]:
# allTransection

In [31]:
from fpgrowth import *

In [32]:
%%time

pattern = find_frequent_patterns( allTransection, 500 )
rules   = generate_association_rules( pattern, 0.8 ) 

print(len(pattern))


20
CPU times: user 14.2 s, sys: 45 ms, total: 14.3 s
Wall time: 14.3 s


Printing most common pattern

In [33]:
pattern

{('1', '1', '1', '1', '1', '1', '1', '1', '1'): 1476,
 ('0', '1', '1', '1', '1', '1', '1', '1', '1', '1'): 11806,
 ('0', '1', '1', '1', '1', '1', '1', '1', '1'): 31896,
 ('0', '0', '1', '1', '1', '1', '1', '1', '1', '1'): 122301,
 ('0', '1', '1', '1', '1', '1', '1', '1'): 69382,
 ('0', '0', '1', '1', '1', '1', '1', '1', '1'): 288708,
 ('0', '1', '1', '1', '1', '1', '1'): 128114,
 ('0', '0', '1', '1', '1', '1', '1', '1'): 589904,
 ('0', '1', '1', '1', '1', '1'): 209682,
 ('0', '0', '1', '1', '1', '1', '1'): 1083432,
 ('0', '1', '1', '1', '1'): 308934,
 ('0', '0', '1', '1', '1', '1'): 1792800,
 ('0', '1', '1', '1'): 403772,
 ('0', '0', '1', '1', '1'): 2612308,
 ('0', '1', '1'): 452454,
 ('0', '0', '1', '1'): 3240201,
 ('0', '1'): 405022,
 ('0', '0', '1'): 3192744,
 ('0',): 175516,
 ('0', '0'): 1464891}

In [34]:
for i in rules:
    print(i, ' => ', rules[i])

('1', '1', '1', '1', '1', '1', '1', '1', '1')  =>  (('0',), 7.9986449864498645)
('0', '1', '1', '1', '1', '1', '1')  =>  ((), 4.604524095727243)
('0', '1', '1', '1', '1', '1', '1', '1')  =>  ((), 4.161136894295351)
('0', '1', '1', '1', '1', '1', '1', '1', '1')  =>  ((), 3.834367945823928)
('0',)  =>  ((), 8.346196358166777)
('0', '1', '1', '1', '1')  =>  ((), 5.80318126201713)
('0', '1', '1', '1', '1', '1')  =>  ((), 5.167024351160329)
('0', '1')  =>  ((), 7.882890312131193)
('0', '1', '1')  =>  ((), 7.161393202402897)
('0', '1', '1', '1')  =>  ((), 6.469760161675401)
('0', '0')  =>  (('1',), 2.1795096017382862)
('0', '0', '1')  =>  ((), 1.0148640166577716)
('0', '0', '1', '1')  =>  ((), 0.8062178858657225)


### Final Comparison

| FP Growth | Apriori|
| ------------- |: ------------- :|
**Pattern Generation**
| FP growth generates pattern by constructing a FP tree | Apriori generates pattern by pairing the items into singletons, pairs and triplets.|
**Candidate Generation**
| There is no candidate generation | Apriori uses candidate generation |
**Process**
The process is faster as compared to Apriori. The runtime of process increases linearly with increase in number of itemsets. | The process is comparatively slower than FP Growth, the runtime increases exponentially with increase in number of itemsets
**Memory Usage**
A compact version of database is saved | The candidates combinations are saved in memory